In [1]:
PATH ="C:\\Users\\GANESH\\Desktop\\pyspark\\dataset2\\"
FILE = "marketing_sample_for_careerbuilder_usa-careerbuilder_job_listing__20200401_20200630__30k_data.ldjson"

#### Dendencies

In [2]:
import pyspark
from pyspark.sql import SparkSession as session
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas
pandas.set_option("display.max.column",None)
import re

#### Create Spark Session

In [3]:
spark = session.builder.appName("job").getOrCreate()

In [4]:
df=spark.read.json(PATH+FILE,primitivesAsString = 'true')

In [5]:
df.printSchema()

root
 |-- city: string (nullable = true)
 |-- company_description: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- contact_email: string (nullable = true)
 |-- country: string (nullable = true)
 |-- crawl_timestamp: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- duplicate_status: string (nullable = true)
 |-- fitness_score: string (nullable = true)
 |-- geo: string (nullable = true)
 |-- has_expired: string (nullable = true)
 |-- html_job_description: string (nullable = true)
 |-- inferred_city: string (nullable = true)
 |-- inferred_country: string (nullable = true)
 |-- inferred_iso2_lang_code: string (nullable = true)
 |-- inferred_iso3_lang_code: string (nullable = true)
 |-- inferred_salary_currency: string (nullable = true)
 |-- inferred_salary_from: string (nullable = true)
 |-- inferred_salary_time_unit: string (nullable = true)
 |-- inferred_salary_to: string (nullable = true)
 |-- inferred_state: string (nullable = true)
 |-- is_

#### Drop duplicates and Unused Columns

In [6]:
df =df.drop('inferred_city','inferred_iso3_lang_code','inferred_salary_time_unit','inferred_iso2_lang_code','inferred_state','last_expiry_check_date','postdate_yyyymmdd')

In [7]:
def dropnul(df):
    null = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    col_to_drop = [key for key, value in null.items() if value > 0.7]
    df = df.drop(*col_to_drop)
    return df
dropnul(df)
df1 = df.transform(dropnul)

calculate number of jobs posted on daily basis, per each city

In [8]:
jobs=df.groupBy(["city"]).agg(
    count("post_date").alias("total_jobs_post"),\
    max("post_date").alias("max_post_date"))
jobs = jobs.sort(desc("total_jobs_post")) 
jobs.show(5)
# jobs.toPandas().to_csv(PATH+"jobs.csv", index=False)

+----------+---------------+-------------+
|      city|total_jobs_post|max_post_date|
+----------+---------------+-------------+
|    Boston|            262|   2020-06-29|
|   Chicago|            218|   2020-06-29|
| Charlotte|            216|   2020-06-26|
|  Portland|            212|   2020-06-28|
|Cincinnati|            212|   2020-06-28|
+----------+---------------+-------------+
only showing top 5 rows



calculate average salary per job title and state

In [9]:
sal =df1.select( 'job_title','state',split("salary_offered","[$/-]").getItem(1).cast('double').alias('salary'))

In [10]:
df2 =sal.groupBy('job_title','state').agg(avg('salary').alias('salary'),count('*'))

In [11]:
avg_salary = df2.na.drop().show(10,False)
avg_salary.toPandas().to_csv(PATH+"avg_salary.csv", index=False)

+------------------------------------------------------+-----+------+--------+
|job_title                                             |state|salary|count(1)|
+------------------------------------------------------+-----+------+--------+
|Sr. Accountant                                        |TX   |30.0  |1       |
|Animal Care Tecnician                                 |NC   |15.0  |1       |
|Traveling Heavy New Store and Store Reset Merchandiser|TX   |14.0  |1       |
|Billing/ Accounting Clerk- Law Firm                   |MN   |20.0  |1       |
|Rebar Fabricator                                      |OH   |15.0  |1       |
|Assembler (T518)                                      |MO   |11.25 |1       |
|Mechanical Assembler                                  |TN   |12.0  |1       |
|2nd shift                                             |MD   |13.1  |1       |
|Customer Service (Technical Support)                  |TN   |11.5  |1       |
|Manual Machinist (Direct Hire! No Contract!)       

AttributeError: 'NoneType' object has no attribute 'toPandas'

Identify the top 10 most active companies by number of positions opened

In [ ]:
company=df.groupBy(["company_name"]).agg(count("post_date").alias("number_of_position"))
company =company.sort(desc("number_of_position"))

# Using split() function

#Convert to Pandas DataFrame
pandasDF=company.limit(10).toPandas()
print(pandasDF)
pandasDF.to_csv(PATH+'Top_10_companies.csv',index=False)

Create a UDF Function

In [ ]:
#create a UDF function to clean job description from HTML code contained inside
# analyze and comment on the performance of the UDF
# function. How the performance could be improved
import time
start_time = time.time()

clean_html =udf(lambda row: re.sub(re.compile('<.*?>'), '',row))
df1 =df1.withColumn('html_job_description', clean_html("html_job_description"))
df1.show()
print(f"Execution time: {time.time() - start_time}")

# Using split() function
df2 = df1.select(split(col("html_job_description"),"Job Description ",2).getItem(1).alias("job1_description")) \
.drop("html_job_description")
df2.show(2)

Pyspark to Pandas

In [ ]:
pdf = df1.toPandas()

In [ ]:
pdf.tail(2)

In [ ]:
# df2.write.format("csv").save(PATH+"analys")